# Bankfull characteristics model: Spatiotemporal evolution of bankfull discharge and channel geometry of an alluvial meandering river. Part 2: Effect of altered flow duration curve on the bankfull channel deformation. 

Kensuke Naito (knaito2@illinois.edu) and Gary Parker

This is a numerical model presented in a paper titled "Can Bankfull Discharge and Bankfull Channel Characteristics of an Alluvial River be Co-specified from the Flow Duration Curve? Part 2: Application to the Minnesota River, USA", which is submitted to Journal of Geophydical Research: Earth Surface in December 2018. The model describes spatiotemporal variation of the bankfull characteristics (bankfull discharge (Qbf), bankfull width (Bbf), bankfull depth (Hbf) and down-channel slope (Sc)) for specified flow duration curve, annual bed material feed rate and other input parameters. From arbitrarily selected initial condition, the model ddepicts the evolution of bankfull characteristics to an equilibrium state, where the variables show no further change both in space and time. 

Here, effect of altered flow duration curve on the bankfull channel deformation is examined for the case of the Minnesota River, MN, USA, whose channel has experienced rapid widening. Total 80 years of discharge record is devided into four periods to construct four flow duratoion curves: FDC1 (1935-1954), FDC2 (1955-1974), FDC3 (1975-1994) and FDC4 (1995-2014). After the model calibration and spin-up run with FDC1 (Regime 0), it is altered to FDC1 (Regime 1), FDC2 (Regime 2), FDC3 (Regime 3) and then FDC4 (Regime 4) with each lasting for 20 years. 

The script is written in Python 2.7. However, it is made in such a way that the model can also be run with PyPy, which is fast, compliant alterative implementation of Python. PyPy can be downloaded at https://pypy.org/.

## 1. Model setup

#### Import libraries

In [ ]:
import os 
import sys
import numpy as np
import csv  

#### Set path and file name

Set path to working directory and the folder where model output csv files will be saved. The file names of the flow duration curve as well as output file need to be specified here. Number at the end of each name refers to the regime: Regime 0 (spin-up), Regime 1 (1935-1954), Regime 2 (1955-1974), Regime 3 (1975-1994) and Regime 4 (1995-2014).

In [ ]:
path_work = 'path to your working directory where fdc files are stored'
path_csv =  'path to directory where you wish to store csv files'
fdc_filename0 = '_fdc_minnesotariver_jordan_1935_1954.csv'
fdc_filename1 = '_fdc_minnesotariver_jordan_1935_1954.csv'
fdc_filename2 = '_fdc_minnesotariver_jordan_1955_1974.csv'
fdc_filename3 = '_fdc_minnesotariver_jordan_1975_1994.csv'
fdc_filename4 = '_fdc_minnesotariver_jordan_1995_2014.csv'
out_filename0 = '_results_regime0.csv'
out_filename1 = '_results_regime1.csv'
out_filename2 = '_results_regime2.csv'
out_filename3 = '_results_regime3.csv'
out_filename4 = '_results_regime4.csv'

#### Input parameters

Most of the input parameters are determined based on USGS gaging station at the Trinity River near Romayor, TX, USA. USGS gaging station number: 05330000.

In [ ]:
# Space / time grid size
dxv = 2000.          # [m] Spatial grid size (valley length)
M = 6          # [1] Number of spatial grids
dt = 0.01          # [yr]  Temporal grid size 
Nloop_0 = 100001   # [1] Numbe of itteration for Regime 0 (spin-up)
Nloop_1234 = 2001   # [1] Number of itteration for Regime 1,2,3,4
au = 0.5          # [1] au = 0.5: Central difference, au = 1: Full upwind

# Initial channel geometry
BbfI = 100.          # [m] Bankfull width   
HncI = 2.          # [m] Thickness of lower floodplain layer
HcI = 2.          #  [m] Thickness of upper floodplain layer
HbfI = HncI + HcI          # [m] Bankfull deoth
ScI = 0.0002164          # [1] Down-channel slope
Etab_d = 10.          # [m] Channel bed elevation at the downstream end 

# Parameter for the flow duration curve
Nfdc = 100          #[1] Number of bins in FDC 

# Calibration parameter
aEH = 0.3          # [1] Engelund-Hansen calibration factor
Tsb = 0.1          # [yr] Characteristic bank armor (slump block) residence time
cveg = 8.          # [m/yr] Characteristic vegetal encroachment rate
fe = 0.2          # [1] Fractional efficiency of floodplain sedimentation

# Auxullary parameter
Cz = 13.          # [1] Dimensionless Chezy channel resistance coeffieent

# Sediment and channel planform
Dbm = 0.3 / 1000.         # [mm] D50 of bed material
QTfeed = 0.22          # [Mt/yr] Mean annual bed material feed rate
Dfm = 0.04 / 1000.          # [mm(m)] floodplain material size (0.05)
Cfm = 0.00015          # [1] Characteristic volumetric concentration of floodplain material
sinu = 2.          # [1] Channel sinuosity

 # Parameter for cut bank erosion and inner bank migration
A = 4.81          # [1] Dimensionless coeff (Johannesson & Parker, 1989)
fmb = 10.          # [1] Ratio of meandering belt width to bankfull width (10)
fc = 1.          # [1] Ratio of slump block size to cohesive layer thickness (0.5)
lpc = 0.3          # [1] Porosity of upper floodplain layer
lp = 0.3          # [1] Porosity of channel bed and lower floodplain layer

# Global constants
g = 9.81          # [m/s^2] Gravitational acceleration
Rr = 1.65          # [1] Submerged specific gravity of sediment
day_in_sec = 60. * 60. * 24.          #[s] A day in seconds
year_in_sec = day_in_sec * 365.25          #[s] A year in seconds 
converter_QT = 1e6 / (Rr + 1.)          # [] Unit converter for bed mateiral load

# Unit conversion
Xv = np.ones(M)          # River valley coordinates for plotting results
for j in range(M):
    Xv[j] *= j * dxv / 1000.
QTfeed *= converter_QT          # [Mt/yr -> m^3/yr] Bed mateiral load

# Fall velocity of floodplain material (Dietrich, 1982)
Rep = np.sqrt(g * Rr * Dfm**3.) / 1e-6          # [1] Particle Reynolds number
b1 = 2.89139447769084; b2 = 0.95296; b3 = 0.0568346711984055 
b4 = 0.00289204602084475; b5 = 0.00024464688411386
Rf = np.exp(-b1 + b2 * np.log(Rep) - b3 * np.log(Rep)**2. - b4 * np.log(Rep)**3. + b5 * np.log(Rep)**4.)
vfall = Rf * np.sqrt(g * Rr * Dfm)          # [m/s] Fall velocity of floodplain material 

# Coefficients for point bar height calculation and cut bank erosion rate calculation
fpb = (sinu - 1.) * A / fmb          # [1] For point bar height calculation
fbe = fc / ((1. - lpc) * Tsb)          # [1/yr] For outer bankf erosion calculation

## 2. Main routine 

This rutine includes initial condition setup, computation of the spatial and temporal bankfull characteristics evolution and result output to csv files. This also includes the alteration of the flow duration curves. First, spin-up run is conduted with FDC1 until it reaches the equilibrium. FDC1 is then switched to FDC1 and run for 20 years (this does not cause channel deformation, it is done for convenience). FDC1 is then altered to FDC2, then run for another 20 years. FDC2 is then changed to FDC3, which is later replaced with FDC4 after 20 years. Thus, total 80 years of model run is conducted after the end of the spin-up run.

In [ ]:
def main():
    
    # Display total number of years of the spin-up calculation and number of years of each regime
    print('Spin-up run [yr] = ' + str(int(dt * Nloop_0))
    print('Regime 1-4 [yr] = ' + str(int(dt * Nloop_1234))
    
    # Change current directory to working directory
    os.chdir(path_work)

    # Initialize the time
    Time = 0.

    # Regime 0 (spin-up) run
    print 'Run: regime 0'
    Stop, Time, Qbf, Hbf, Bbf, Sc, Hc, Hnc, Hpb, \
    Etab, Etac, Etanc, Etapb, cE, cD, vD, QT \
          = regime_0(Time)

    # Regime 1-4 runs
    for regime in range(1, 5):
        if Stop == True:
            break
        else:
            print 'Run: Regime ' + str(int(regime))
            Stop, Time, Qbf, Hbf, Bbf, Sc, Hc, Hnc, Hpb, \
            Etab, Etac, Etanc, Etapb, cE, cD, vD, QT \
                  = regime_1234(regime, Time, Qbf, Hbf, Bbf, Sc,
                                Hc, Hnc, Hpb, Etab, Etac, Etanc, Etapb,
                                cE, cD, vD, QT)
    
    # Display the main results
    if Stop == False:
        print('===== MAIN RESULTS (at the end of Regime 4) =====')
        print('Qbf [m^3/s] = ' + str(round(Qbf[0], 1)))
        print('Bbf [m] = ' + str(round(Bbf[0], 1)))
        print('Hbf [m] = ' + str(round(Hbf[0], 1)))

## 3. Sub-routines for Regime 0 run and Regime 1-4 runs

#### Regime 0 (spin-up run)

In [ ]:
def regime_0(Time):
    
    # Setup empty arrays for result output to csv file 
    TIME = []; QBF = []; HBF = []; BBF = []; SC = []
    HC = []; HNC = []; HPB = []
    ETAB = []; ETAC = []; ETANC = []; ETAPB = []
    CERO = []; CDEP = []; VDEP = []; QTT = []

    # Constract FDC from csv file on daily-averge flow record
    Qi, pi = construct_fdc(fdc_filename0)

    # Calculate and setup initial condition
    usbf = (g * HbfI * ScI)**(1./2.)
    QbfI = Cz * usbf * HbfI * BbfI
    HpbI = fpb * HbfI
    cEI = outerbank_erosion_rate(QbfI, BbfI, HbfI, HncI, ScI, Qi, pi)
    cDI = innerbank_deposition_rate(QbfI, BbfI, HbfI, ScI, Qi, pi)
    vDI = overbank_deposition_rate(QbfI, Qi, pi)
    QTI = bedmaterial_load(QbfI, BbfI, HbfI, ScI, Qi, pi)

    # Set up the spatial grids for initial condition
    Qbf = QbfI * np.ones(M); Bbf = BbfI * np.ones(M)
    Hbf = HbfI * np.ones(M); Sc = ScI * np.ones(M)
    Hnc = HncI * np.ones(M); Hc = HcI * np.ones(M)
    Hpb = HpbI * np.ones(M)
    Etab = Etab_d * np.ones(M)
    for j in range(M):
        Etab[j] += ScI * sinu * dxv * (M - j - 1.)
    Etanc = Etab + HncI; Etac = Etab + HbfI
    Etapb = Etab + HpbI
    cE = cEI * np.ones(M); cD = cDI * np.ones(M)
    vD = vDI * np.ones(M); QT = QTI * np.ones(M)

    # Temporal loop
    for k in range(Nloop_0):
        
        # Advance the clock
        Time += dt
        
        # Spatial evolution of the variables
        Stop, Qbf, Hbf, Bbf, Sc, Hc, Hnc, Hpb, \
              Etab, Etac, Etanc, Etapb, cE, cD, vD, QT  \
              = spatial_calculation(Qbf, Bbf, Sc, Hnc, Hpb,
                                    Etac, Etanc, Etab, Etapb,
                                    cE, cD, vD, QT, QTfeed, Qi, pi)
                
        # Update lists
        if Stop == True:
            break
        else:
            TIME.append(Time); QBF.append(np.mean(Qbf)); HBF.append(np.mean(Hbf))
            BBF.append(np.mean(Bbf)); SC.append(np.mean(Sc))
            HC.append(np.mean(Hc)); HNC.append(np.mean(Hnc)); HPB.append(np.mean(Hpb))
            ETAB.append(np.mean(Etab)); ETAC.append(np.mean(Etac))
            ETANC.append(np.mean(Etanc)); ETAPB.append(np.mean(Etapb))
            CERO.append(np.mean(cE)); CDEP.append(np.mean(cD))
            VDEP.append(np.mean(vD)); QTT.append(np.mean(QT))

    # Print out results to csv file
    if Stop != True:
        printout_csv(out_filename0, np.array(TIME), np.array(QBF), np.array(HBF),
                     np.array(BBF), np.array(SC),
                     np.array(HC), np.array(HNC), np.array(HPB),
                     np.array(ETAB), np.array(ETAC), np.array(ETANC), np.array(ETAPB),
                     np.array(CERO), np.array(CDEP), np.array(VDEP), np.array(QTT))

    return Stop, Time, Qbf, Hbf, Bbf, Sc, Hc, Hnc, Hpb, \
           Etab, Etac, Etanc, Etapb, cE, cD, vD, QT

#### Regime 1-4

In [ ]:
def regime_1234(regime, Time, Qbf, Hbf, Bbf, Sc,
                Hc, Hnc, Hpb, Etab, Etac, Etanc, Etapb,
                cE, cD, vD, QT):

   # Setup empty arrays for result output to csv file 
    TIME = []; QBF = []; HBF = []; BBF = []; SC = []
    HC = []; HNC = []; HPB = []
    ETAB = []; ETAC = []; ETANC = []; ETAPB = []
    CERO = []; CDEP = []; VDEP = []; QTT = []

    # Set file names and parameters for each regime
    if regime == 1:
        fdc_filename = fdc_filename1
        out_filename = out_filename1
    elif regime == 2:
        fdc_filename = fdc_filename2
        out_filename = out_filename2
    elif regime == 3:
        fdc_filename = fdc_filename3
        out_filename = out_filename3
    else:
        fdc_filename = fdc_filename4
        out_filename = out_filename4

    # Constract FDC from csv file on daily-averge flow record
    Qi, pi = construct_fdc(fdc_filename)

    # Temporal loop
    for k in range(Nloop_1234):
        
        # Advance the clock
        Time += dt

        # Spatial evolution of the variables
        Stop, Qbf, Hbf, Bbf, Sc, Hc, Hnc, Hpb, \
        Etab, Etac, Etanc, Etapb, Cero, Cdep, Vdep, Qt \
            = spatial_calculation(Qbf, Bbf, Sc, Hnc, Hpb, Etac, Etanc, Etab, Etapb,
                                  cE, cD, vD, QT, QTfeed, Qi, pi)

        # Update lists
        if Stop == True:
            break
        else:
            TIME.append(Time); QBF.append(np.mean(Qbf)); HBF.append(np.mean(Hbf))
            BBF.append(np.mean(Bbf)); SC.append(np.mean(Sc))
            HC.append(np.mean(Hc)); HNC.append(np.mean(Hnc)); HPB.append(np.mean(Hpb))
            ETAB.append(np.mean(Etab)); ETAC.append(np.mean(Etac))
            ETANC.append(np.mean(Etanc)); ETAPB.append(np.mean(Etapb))
            CERO.append(np.mean(cE)); CDEP.append(np.mean(cD))
            VDEP.append(np.mean(vD)); QTT.append(np.mean(QT))

    # Print out results to csv file
    if Stop != True:
        printout_csv(out_filename, np.array(TIME), np.array(QBF), np.array(HBF),
                     np.array(BBF), np.array(SC),
                     np.array(HC), np.array(HNC), np.array(HPB),
                     np.array(ETAB), np.array(ETAC), np.array(ETANC), np.array(ETAPB),
                     np.array(CERO), np.array(CDEP), np.array(VDEP), np.array(QTT))

    return Stop, Time, Qbf, Hbf, Bbf, Sc, Hc, Hnc, Hpb, \
           Etab, Etac, Etanc, Etapb, cE, cD, vD, QT

## 4. Sub-functions

#### Spatial variation of the bankfull characteristics 

Bankfull channel characteristics are calculated as a result of floodplain thickening due to overbank deposition or/and channel bed degradation, floodplain thinning due to cut bank erosion or/and channel bed aggracation, channel widening/narrowing due to the difference in cut bank migraion rate and inner bank migration rate. Altough spatial variation of the parameters are calculated, they are not varying over space sice channel slope is fixed and uniform in space. Qtfeed does, thus, not play a role herein. 

In [ ]:
def spatial_calculation(Qbf, Bbf, Sc, Hnc, Hpb, Etac, Etanc, Etab, Etapb,
                        cE, cD, vD, QT, QTfeed, Qi, pi):

    # Create arrays
    d_Etab = np.zeros(M); d_Etanc = np.zeros(M)
    d_Etac = np.zeros(M); d_Bbf = np.zeros(M)

    # Calculate update
    for j in range(M):
        # QT at adjacent nodes
        QTmid = QT[j]
        if j == 0:
            QTup = QT[j]          # Note that QTfeed does not play a role herein
            QTdw = QT[j+1]
        elif j == M-1:
            QTup = QT[j-1]
            QTdw = QTmid
        else:
            QTup = QT[j-1]
            QTdw = QT[j+1]

        d_Etab[j] = (Hnc[j] * cE[j] - Hpb[j] * cD[j]) / Bbf[j] \
                    - (au * (QTmid - QTup) + (1. - au) * (QTdw - QTmid)) \
                    / (dxv * sinu * (1. - lp) * Bbf[j])
        d_Etanc[j] = (Etapb[j] - Etanc[j]) * sinu * cD[j] / (fmb * Bbf[j] - sinu * Bbf[j])
        d_Etac[j] = vD[j] / (1. - lpc) + sinu * (cD[j] * (Etapb[j] - Etanc[j]) - cE[j] * (Etac[j] - Etanc[j])) \
                    / (fmb * Bbf[j] - sinu * Bbf[j])
        d_Bbf[j] = cE[j] - cD[j]

    # Update geometry
    Bbf = Bbf + dt * d_Bbf
    Etab = Etab + dt * d_Etab
    Etanc = Etanc + dt * d_Etanc
    Etac = Etac + dt * d_Etac
    Hnc = Etanc - Etab
    Hc = Etac - Etanc
    Hbf = Hnc + Hc
    Hpb = fpb * Hbf
    Etapb = Etab + Hpb
        
    # Calculate bankfull discharge, channel migration, 
    # overbank deposition and bed material load
    for j in range(M):
        # Detect NAN and negative geometry
        if Bbf[j] != Bbf[j] or QT[j] != QT[j] or Bbf[j] <= 0 or Sc[j] <= 0:
            Stop = True
            print '[!] Encountered NAN or negative geometry. Change dt, dx or initial condition'
            break
        else:
            Stop = False
            # Bankfull discharge
            usbf = (g * Hbf[j] * Sc[j])**(1./2.)
            Qbf[j] = Cz * usbf * Hbf[j] * Bbf[j]
            # Lateral channel migration, overbank deposition and bed mateiral load
            cE[j] = outerbank_erosion_rate(Qbf[j], Bbf[j], Hbf[j], Hnc[j], Sc[j], Qi, pi)  # [m/yr]
            cD[j] = innerbank_deposition_rate(Qbf[j], Bbf[j], Hbf[j], Sc[j], Qi, pi)  # [m/yr]
            vD[j] = overbank_deposition_rate(Qbf[j], Qi, pi)  # [m/yr]
            QT[j] = bedmaterial_load(Qbf[j], Bbf[j], Hbf[j], Sc[j], Qi, pi)  # [m^3/yr]
    
    return Stop, Qbf, Hbf, Bbf, Sc, Hc, Hnc, Hpb, \
           Etab, Etac, Etanc, Etapb, cE, cD, vD, QT

#### Function to calculate inner bank depositional migration

Inner bank depositional migration rate (cD) is calculated as a negative function of specified characteristic point bar vegetal encroachment rate (cveg) and bed shear stress (taus). Note that cD is integrated over the specified flow duration curve.  

In [ ]:
def innerbank_deposition_rate(Qbf, Bbf, Hbf, Sc, Qi, pi):
    tausbf = Hbf * Sc / (Rr * Dbm)
    cD = 0.
    for i in range(Nfdc):
        # No inner bank advance during overbank flows
        if Qi[i] <= Qbf:
            H = (Qi[i]**2. / (Cz**2. * g * Sc * Bbf**2.))**(1./3.)
            tausi = H * Sc / (Rr * Dbm)
            cD += 0.5 * cveg / (1. - lp) * (1. - tausi / tausbf) * pi[i]
    return cD

#### Cut bank erosional migration 

Outer bank erosional migration rate (cE) is calculated as a function of specified characteristic bank aromor (slump block) residence time (Tsb) and bed shear stress (taus). Note that cE is integrated over the specified flow duration curve.  

In [ ]:
def outerbank_erosion_rate(Qbf, Bbf, Hbf, Hnc, Sc, Qi, pi):
    if Hnc < 0.:
        Hnc = 0.
    tausbf = Hbf * Sc / (Rr * Dbm)
    cE = 0.
    for i in range(Nfdc):
        # Below bankfull flow
        if Qi[i] <= Qbf:
            H = (Qi[i]**2. / (Cz**2. * g * Sc * Bbf**2.))**(1./3.)
            tausi = H * Sc / (Rr * Dbm)
            cE += 0.5 * fbe * Hnc * (tausi / tausbf) * pi[i]
        # Above bankfull flow
        else:
            cE += 0.5 * fbe * Hnc * 1. * pi[i]
    return cE

#### Overbank deposition 

Overbank deposition rate (vD) is calculated as a function of fall velocity of specified fractional efficiency of floodplain mateiral sedimentation (fe), specified characteristic volume concentration of floodplain mateiral during overbank flows (Cfm) and settling velocity of specified floodplain mateiral grain size (vfall). Note that vD is integrated over the specified flow duration curve.  

In [ ]:
def overbank_deposition_rate(Qbf, Qi, pi):
    vdep = 0.
    for i in range(Nfdc):
        # No overbank deposition rate during below-bankfull flows
        if Qi[i] > Qbf:
            vdep += fe * vfall * year_in_sec * Cfm * pi[i]
    return vdep

#### In-channel bed mateiral transport

In-channel bedmaterial load (QT) is calculated using total load relation by Engelund and Hansen (1967). It is calculated as a function of specified total load relation calibration factor (aEH), bed shear stress (taus) and specified dimensionless Chezy channel resistance coefficieit (Cz). Note that QT is integrated over the specified flow duration curve.

In [ ]:
def bedmaterial_load(Qbf, Bbf, Hbf, Sc, Qi, pi):
    tausbf = Hbf * Sc / (Rr * Dbm)
    Einstein_no = 0.
    for i in range(Nfdc):
        # Below overbank flow
        if Qi[i] <= Qbf:
            H = (Qi[i]**2. / (Cz**2. * g * Sc * Bbf**2.))**(1./3.)
            tausi = H * Sc / (Rr * Dbm)
            Einstein_no += Bbf * aEH * 0.05 * Cz**2. * tausi**(5./2.) * pi[i]
        # Above overbank flow
        else:
            Einstein_no += Bbf * aEH * 0.05 * Cz**2. * tausbf**(5./2.) * pi[i]
    QT = Einstein_no * (g * Rr * Dbm**3.)**(1./2.) * year_in_sec          # [m^3/yr]
    return QT

#### Read csv file to condtruct the flow duration curve

The cvs file containes 4 columns (1: agency, 2: site no, 3: date, 4: discharge). Discharge is provided in US unit, thus it is converted to SI unit. 

In [ ]:
def construct_fdc(fdc_filename):
    # Read csv file
    with open(fdc_filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        Qw = []; counter = 0
        for row in csv_reader:
            if str(row[3]).isdigit() and str(row[3]) > 0.:
                Qw.append(float(row[3]) * (12.**3.) * (2.54**3.) / (100.**3.))     #[ft^3/s -> m^3/s]
            counter += 1
    Qw = np.array(Qw)
    # Condtruct probability density function
    Qi, pi = np.zeros(Nfdc), np.zeros(Nfdc)
    nn, bins = np.histogram(Qw, Nfdc, density=True)
    for k in range(Nfdc):
        Qi[k] = 0.5 * (bins[k] + bins[k+1])
        pi[k] = nn[k] * (bins[k+1] - bins[k])
    return Qi, pi

#### Print out results to csv files

Each output file containes temporal variation of time (TIME), bankfull discharge (Qbf), bankfull width (Bbf), bankful depth (Hbf = Hnc + Hc)), down-channel slope (Sc), elevation of channel bed (Etab), elevation of top of upper cohesive floodplain layer (Etac), elevation of top of lower non-cohesive floodplain layer (Etanc), elevation of top of point bar (Etapb), thichness of upper cohesive floodplain layer (Hc = Etac - Etanc), thickness of lower non-cohesive floodplain layer (Hnc = Etanc - Etab), point bar height (Hpb = Etapb - Etab), cut bank erosional migraion rate (cE), inner bankf depositional migraion rate (cD), overbank deposition rage (vD) and annual bed material load (QT). Note that these variables do not change over space.

In [ ]:
def printout_csv(out_filename, TIME, QBF, HBF, BBF, SC, HC, HNC, HPB,
                 ETAB, ETAC, ETANC, ETAPB, CERO, CDEP, VDEP, QTT):
    # Change the directory
    os.chdir(path_csv)
    Nrow = len(TIME)
    QTT /= converter_QT     # [m^3/yr -> Mt/yr]
    with open(out_filename, 'wb') as ff:
        writer = csv.writer(ff)
        for kk in range(Nrow):
            # [0: TIME, 1:QBF, 2:HBF, 3:BBF, 4:SC,
            #  5:ETAB, 6:ETAC, 7:ETANC, 8:ETAPB, 
            #  9:HC, 10:HNC, 11:HPB, 12:CERO, 13:CDEP, 14:VDEP, 15:QTT]
            writer.writerow([
                '{:.1f}'.format(TIME[kk]), '{:.3f}'.format(QBF[kk]),
                '{:.3f}'.format(HBF[kk]), '{:.3f}'.format(BBF[kk]),
                '{:.8f}'.format(SC[kk]),
                '{:.3f}'.format(ETAB[kk]), '{:.3f}'.format(ETAC[kk]),
                '{:.3f}'.format(ETANC[kk]), '{:.3f}'.format(ETAPB[kk]),
                '{:.3f}'.format(HC[kk]), '{:.3f}'.format(HNC[kk]),
                '{:.3f}'.format(HPB[kk]),
                '{:.3f}'.format(CERO[kk]), '{:.3f}'.format(CDEP[kk]),
                '{:.10f}'.format(VDEP[kk]), '{:.3f}'.format(QTT[kk])])
    # Change the directory back
    os.chdir(path_work)

## 5. Run the script

In [ ]:
if __name__ == '__main__':
    main()